In [1]:
#Downloaing the necessary libraries
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.14.0
!pip install opencv-python
!pip install imutils

In [2]:
#importint the necessary lirbraries

#import tensorflow.keras as kerasfrom __future__ import print_function
from __future__ import print_function

import keras
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



import tensorflow as tf
from keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import utils
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam, SGD
from keras import optimizers
from keras.models import model_from_json
from keras.layers import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from os import listdir
import os
from imutils import paths
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
import random

In [2]:
keras.__version__, tf.__version__

('2.2.4', '1.14.0')

In [6]:
#helper functions
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x

def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)

def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)

class Capsule(Layer):
    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

            

In [7]:
#defining the network
#building the model
input_image = Input(shape=(224, 224, 3))

'''
x = Conv2D(64, (3, 3), activation='relu')(input_image)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = AveragePooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = Reshape((-1, 128))(x)
capsule = Capsule(10, 16, 3, True)(x)
'''

x = Convolution2D(96, 11, strides = (4,4), padding = 'valid', input_shape = (224,224,3), activation = 'relu')(input_image)
x = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid")(x)
x = BatchNormalization()(x)

#second convolution set
x = Convolution2D(256, 11, strides = (1,1), padding = "valid", activation = "relu")(x)
x = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid")(x)
x = BatchNormalization()(x)

#third convolution set
x = Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu")(x)
x = BatchNormalization()(x)

#fourth convolution set
x = Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu")(x)
x = BatchNormalization()(x)

#fifth convolution set
x = Convolution2D(256, 3, strides = (1,1), padding = "valid", activation = "relu")(x)
x = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid")(x)
x = BatchNormalization()(x)

#flattening step
x = Flatten()(x)

#dense network
x = Dense(units = 4096, activation = "relu")(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)

x = Dense(units = 4096, activation = "relu")(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)

x = Dense(units = 1000, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)

x = Reshape((-1, 1000))(x)
capsule = Capsule(10, 16, 3, True)(x)
output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(capsule)
classifier = Model(inputs=input_image, outputs=output)

In [8]:
# serialize model to JSON
model_json = classifier.to_json()
with open("model_classify.json", "w") as json_file:
    json_file.write(model_json)
    

In [ ]:
class

In [10]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 8, 8, 256)         1024      
__________

In [5]:
#Now rewriting the algorithm to preprocess about 500 images in my own pipeline
#only do this if you have enough RAM
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, (224, 224))
            return img_to_array(image)
        else:
            return np.array([])
        
    except Exception as e:
        print(f"Error : {e}")
        return None

image_list = np.array([])

directory_root = '../dataset/train'
imagePaths = list(paths.list_images(directory_root))
imagePaths = imagePaths[0:-1:6]
random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-2] for p in imagePaths]

batch_size = 16
try:
    for i in np.arange(0, len(imagePaths), batch_size):
        batchPaths = imagePaths[i:i + batch_size]
        batchLabels = labels[i: i + batch_size]
        
        for image in batchPaths:
            image_list.append(convert_image_to_array(image))
except Exception as e:
    print(f"Error: {e}")


len(image_list), len(labels)

np_image_list = np.array(image_list, dtype = np.float16) / 255.0
print(f"[INFO] splitting data")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, labels, test_size = 0.2, random_state = 42)

In [10]:
classifier.load_weights('../models/best_weights_capsule.hdf5')
classifier.save("../models/test_capsule_model.hdf5")

In [ ]:
weight_path = '../models/best_weights_capsule.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_accuracy', verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]
classifier.fit(x_train, y_train, batch_size = 8, epochs = 30, validation_data=(x_test, y_test), shuffle=True, callbacks=callbacks_list)

In [14]:
#now to  build a custom pipeline
classifier.compile(optimizer = optimizers.SGD(lr=0.001, momentum = 0.9, decay = 0.005), loss=margin_loss, metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale= 1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
base_dir = "../dataset"

training_set = train_datagen.flow_from_directory(base_dir+'/train', target_size=(224,224),batch_size = batch_size, class_mode='categorical')
valid_set = valid_datagen.flow_from_directory(base_dir+'/valid', target_size=(224,224),batch_size = batch_size, class_mode='categorical')

class_dict = training_set.class_indices
li = list(class_dict)

training_num = training_set.samples
valid_num = valid_set.samples

weight_path = '../models/best_weights_capsule.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=False, mode='max')
callbacks_list = [checkpoint]

history = classifier.fit_generator(training_set, steps_per_epoch=training_num//batch_size, epochs=100,validation_data=valid_set, validation_steps=valid_num//batch_size, callbacks=callbacks_list)


Found 18160 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_16/Relu}}]]
	 [[loss/mul/_1019]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_16/Relu}}]]
0 successful operations.
0 derived errors ignored.